In [5]:
using HDF5, EzXML, Glob, Dates, Plots, Optim, Symbolics, JuMP, StatsBase, DataFrames, CSV, Random, Distributions, Printf, GLM
using CurveFit
include("/Users/jjc/CSF/CSF Dynamics Scripts/readCSF.jl")

datapath = "/Users/jjc/CSF/Recordings/"
path = pwd();
savepath = "/Users/jjc/CSF/"
files = glob("*.hdf5", datapath);

global df_static = DataFrame(filename=files, Rcsf=NaN, Ib=NaN, P0=NaN, R2=NaN)
global df_R = DataFrame(filename=files, Rcsf=NaN, Ib=NaN, P0=NaN, R2=NaN)
global df_RP = DataFrame(filename=files, Rcsf=NaN, Ib=NaN, P0=NaN, R2=NaN)

j = 1

# for j = 1:length(files)
#     try
filename = files[j]
global Data = readCSF(filename)

icp = Data["ICP"]
st = Data["infusion_start_frame"]
en = Data["infusion_end_frame"]

Pm = icp[st:en]

trans_st = Data["transition_start_frame"]
trans_en = Data["transition_end_frame"]
icp = Data["ICP"][trans_st:trans_en]
amp = Data["AMP"][trans_st:trans_en]
rm = .~isnan.(amp)
rm = rm .&& amp .> 0.1
amp = amp[rm]
icp = icp[rm]
x = icp
y = amp
reg = lm(@formula(y ~ x), DataFrame(x=icp, y=amp))
R2_old = r2(reg)
residuals = y .- GLM.predict(reg, DataFrame(x=icp, y=amp))
std_res = residuals ./ std(residuals)
keep = abs.(std_res) .<= 1
reg_cleaned = lm(@formula(y ~ x), DataFrame(x=icp[keep], y=amp[keep]))
c, a = GLM.coeftable(reg_cleaned).cols[1, 1]
R2 = r2(reg_cleaned)
y_intercept = -c / a
x_values = range(y_intercept, maximum(icp), length=100)
y_values = x_values .* a .+ c

P0 = y_intercept
Rcsf = (Data["P_p"] - Data["P_b"]) / Data["I_inf"]

volRes, pressRes, fitted_curve, R2, SSE = press_vol_curve(Rcsf, P0, Pm)

bgcolor = :white
plot(volRes, log.(pressRes), seriestype=:scatter, makersize=5, color=:black, markerstrokewidth=0.5, markerstrokecolor=:white, linewidth=1, label="Bayesian", grid=true, titlefontsize=8, titlealign=:left, background=bgcolor, xlabel="Volume [mL]", ylabel="(P - Pss)/(Pb - Pss) [1/1]", yguidefontsize=8, xguidefontsize=8)
plot!(volRes, fitted_curve, lw=2, linecolor=:black, linestyle=:dash, legend=:outertopright, label="")
title!(@sprintf("R² = %0.2f", R2))

df_static.Rcsf[j] = round(Rcsf, digits=2)
df_static.R2[j] = round(R2, digits=2)
df_static.P0[j] = round(P0, digits=2)
df_static.Ib[j] = round(((Data["P_b"] - P0) / Rcsf), digits=2)

print(@sprintf("Rcsf = %0.2f\n", Rcsf))
print(@sprintf("P₀ (static) = %0.2f\n", P0))
print(@sprintf("R² = %0.3f\n", R2))
print(@sprintf("Iₐ = %0.2f", ((Data["P_b"] - P0) / Rcsf)))





#######################################

filename = files[j]
icp = Data["ICP"]
st = Data["infusion_start_frame"]
en = Data["infusion_end_frame"]
Pm = icp[st:en]

trans_st = Data["transition_start_frame"]
trans_en = Data["transition_end_frame"]
icp = Data["ICP"][trans_st:trans_en]
amp = Data["AMP"][trans_st:trans_en]
rm = .~isnan.(amp)
rm = rm .&& amp .> 0.1
amp = amp[rm]
icp = icp[rm]
x = icp
y = amp
reg = lm(@formula(y ~ x), DataFrame(x=icp, y=amp))
R2_old = r2(reg)
residuals = y .- GLM.predict(reg, DataFrame(x=icp, y=amp))
std_res = residuals ./ std(residuals)
keep = abs.(std_res) .<= 1
reg_cleaned = lm(@formula(y ~ x), DataFrame(x=icp[keep], y=amp[keep]))
c, a = GLM.coeftable(reg_cleaned).cols[1, 1]
R2 = r2(reg_cleaned)
y_intercept = -c / a
x_values = range(y_intercept, maximum(icp), length=100)
y_values = x_values .* a .+ c

R = LinRange(1, 50, 100000)
# P0 = Data["P_0"]
P0 = y_intercept

SSE = zeros(length(R))
R2 = zeros(length(R))

for i = 1:length(R)
    Rcsf = R[i]
    SSE[i] = press_vol_curve(Rcsf, P0, Pm)[5]
    R2[i] = press_vol_curve(Rcsf, P0, Pm)[4]
end

rcsf = R[findmin(SSE)[2]]
p0 = P0
Ib = (Data["P_b"] - p0) / rcsf

print(@sprintf("Rcsf = %0.2f\n", rcsf))
print(@sprintf("P₀ (static) = %0.2f\n", P0))
print(@sprintf("R² = %0.3f\n", R2[findmin(SSE)[2]]))
print(@sprintf("Iₐ = %0.2f", ((Data["P_b"] - p0) / rcsf)))

df_R.Rcsf[j] = round(rcsf, digits=2)
df_R.R2[j] = round(R2[findmin(SSE)[2]], digits=2)
df_R.P0[j] = round(P0, digits=2)
df_R.Ib[j] = round(((Data["P_b"] - p0) / rcsf), digits=2)


##########################################

R = LinRange(1, 50, 400)
P0 = LinRange(-10, Data["P_b"], 400)

SSE = zeros(length(R), length(P0))
R2 = zeros(length(R), length(P0))

for i = 1:length(R)
    for j = 1:length(P0)
        Rcsf = R[i]
        P_0 = P0[i]
        R2[i, j], SSE[i, j] = press_vol_curve(Rcsf, P_0, Pm)[4:5]
    end
end

SSE = SSE[.~isnan.(SSE)]
R2 = R2[.~isnan.(R2)]

P0[findmin(SSE)[2]]

rcsf = R[findmin(SSE)[2]]
p0 = P0[findmin(SSE)[2]]
Ib = (Data["P_b"] - p0) / rcsf

print(@sprintf("Rcsf = %0.2f\n", rcsf))
print(@sprintf("P₀ = %0.2f\n", p0))
print(@sprintf("R² = %0.3f\n", R2[findmin(SSE)[2]]))
print(@sprintf("Iₐ = %0.2f", Ib))

df_RP.Rcsf[j] = round(rcsf, digits=2)
df_RP.R2[j] = round(R2[findmin(SSE)[2]], digits=2)
df_RP.P0[j] = round(p0, digits=2)
df_RP.Ib[j] = round(Ib, digits=2)
#     catch
#         continue
#     end

# end

# CSV.write("/Users/jjc/CSF/results_feb2023/" * "PV_static.csv", df_static)
# CSV.write("/Users/jjc/CSF/results_feb2023/" * "PV_R.csv", df_R)
# CSV.write("/Users/jjc/CSF/results_feb2023/" * "PV_RP.csv", df_RP)



Rcsf = 11.20
P₀ (static) = 7.69
R² = 0.632
Iₐ = 1.01Rcsf = 50.00
P₀ (static) = 7.69
R² = 0.879
Iₐ = 0.23Rcsf = 1.00
P₀ = -10.00
R² = 0.869
Iₐ = 29.04

29.04

In [46]:
# bruteforce: just Rcsf

for j = 3
    filename = files[j]
    icp = Data["ICP"]
    st = Data["infusion_start_frame"]
    en = Data["infusion_end_frame"]
    Pm = icp[st:en]
    
trans_st = Data["transition_start_frame"]
trans_en = Data["transition_end_frame"]
icp = Data["ICP"][trans_st:trans_en]
amp = Data["AMP"][trans_st:trans_en]
rm = .~isnan.(amp)
rm = rm .&& amp .> 0.1
amp = amp[rm]
icp = icp[rm]
x = icp
y = amp
reg = lm(@formula(y ~ x), DataFrame(x=icp, y=amp))
R2_old = r2(reg)
residuals = y .- GLM.predict(reg, DataFrame(x=icp, y=amp))
std_res = residuals ./ std(residuals)
keep = abs.(std_res) .<= 1
reg_cleaned = lm(@formula(y ~ x), DataFrame(x=icp[keep], y=amp[keep]))
c, a = GLM.coeftable(reg_cleaned).cols[1, 1]
R2 = r2(reg_cleaned)
y_intercept = -c / a
x_values = range(y_intercept, maximum(icp), length=100)
y_values = x_values .* a .+ c

R =LinRange(1, 50, 100000)
# P0 = Data["P_0"]
P0 = y_intercept

SSE = zeros(length(R))
R2 = zeros(length(R))

for i = 1:length(R)
    Rcsf = R[i]
    SSE[i] = press_vol_curve(Rcsf, P0, Pm)[5]
    R2[i] = press_vol_curve(Rcsf, P0, Pm)[4]
end

rcsf = R[findmin(SSE)[2]]
p0 =  P0
Ib = (Data["P_b"] - p0)/rcsf
# print(@sprintf("Rcsf = %0.2f\n", R[findmin(SSE)[2]]))
# print(@sprintf("P₀ (static) = %0.2f\n", P0))
# print(@sprintf("R² = %0.3f\n", R2[findmin(SSE)[2]]))
# print(@sprintf("Iₐ = %0.2f", Ib))

df.Rcsf[j] = rcsf
df.R2[j] = R2[findmin(SSE)[2]]
df.P0[j] = P0
df.Ib[j] = (Data["P_b"] - p0)/rcsf
end

df


,filename,Rcsf,Ib,P0
,String,Float64,Float64,Float64
1,/Users/jjc/CSF/Recordings/031114DP.hdf5,NaN,NaN,NaN
2,/Users/jjc/CSF/Recordings/2003-09-10_MT.hdf5,NaN,NaN,NaN
3,/Users/jjc/CSF/Recordings/2003-10-10_PT.hdf5,5.12584,0.841811,7.47501
4,/Users/jjc/CSF/Recordings/2003-11-07_NG.hdf5,NaN,NaN,NaN
5,/Users/jjc/CSF/Recordings/2003-11-10_LH.hdf5,NaN,NaN,NaN
6,/Users/jjc/CSF/Recordings/2003-11-21_RP.hdf5,NaN,NaN,NaN
7,/Users/jjc/CSF/Recordings/2003-11-24_EP.hdf5,NaN,NaN,NaN
8,/Users/jjc/CSF/Recordings/2003-11-25_JF.hdf5,NaN,NaN,NaN
9,/Users/jjc/CSF/Recordings/2003-12-04_JF.hdf5,NaN,NaN,NaN


In [41]:
# bruteforce Rcsf + P0

R = LinRange(1, 50, 400)
P0 = LinRange(-10, Data["P_b"], 400)

SSE = zeros(length(R), length(P0))
R2 = zeros(length(R), length(P0))

for i = 1:length(R)
    for j = 1:length(P0)
        Rcsf = R[i]
        P_0 = P0[i]
        R2[i,j], SSE[i,j] = press_vol_curve(Rcsf, P_0, Pm)[4:5]
    end
end

SSE = SSE[.~isnan.(SSE)]
R2 = R2[.~isnan.(R2)]

P0[findmin(SSE)[2]]

rcsf = R[findmin(SSE)[2]]
p0 =  P0[findmin(SSE)[2]]
Ib = (Data["P_b"] - p0)/rcsf
# print(@sprintf("Rcsf = %0.2f\n", rcsf))
# print(@sprintf("P₀ = %0.2f\n", p0))
# print(@sprintf("R² = %0.3f\n", R2[findmin(SSE)[2]]))
# print(@sprintf("Iₐ = %0.2f", Ib))

df.Rcsf[j] = rcsf
df.R2[j] = R2[findmin(SSE)[2]]
df.P0[j] = p0
df.Ib[j] = Ib


,filename,Rcsf,Ib,P0
,String,Float64,Float64,Float64
1,/Users/jjc/CSF/Recordings/031114DP.hdf5,NaN,NaN,NaN
2,/Users/jjc/CSF/Recordings/2003-09-10_MT.hdf5,NaN,NaN,NaN
3,/Users/jjc/CSF/Recordings/2003-10-10_PT.hdf5,1.0,21.79,-10.0
4,/Users/jjc/CSF/Recordings/2003-11-07_NG.hdf5,NaN,NaN,NaN
5,/Users/jjc/CSF/Recordings/2003-11-10_LH.hdf5,NaN,NaN,NaN
6,/Users/jjc/CSF/Recordings/2003-11-21_RP.hdf5,NaN,NaN,NaN
7,/Users/jjc/CSF/Recordings/2003-11-24_EP.hdf5,NaN,NaN,NaN
8,/Users/jjc/CSF/Recordings/2003-11-25_JF.hdf5,NaN,NaN,NaN
9,/Users/jjc/CSF/Recordings/2003-12-04_JF.hdf5,NaN,NaN,NaN


In [97]:
h7 = scatter(Data["ICP"][2:end], Data["AMP"][2:end], color=:green, alpha=0.1, label="Data", markerstrokecolor=:transparent)
scatter!(x[.~keep], y[.~keep], color=:red, label="Outliers", markerstrokecolor=:transparent)
scatter!(x[keep], y[keep], color=:black, alpha=1.0, label="Transition", markerstrokecolor=:transparent)
plot!(x_values, y_values, label="fit", legend=:outertopright, ylims=[0, :auto], xlabel="ICP [mmHg]", ylabel="AMP [mmHg]", linewidth=2)
title!(@sprintf("R² noisy = %0.2f\nR² denoised = %0.2f", R2_old, R2), titlefontsize=8, titlealign=:left, yguidefontsize=8, xguidefontsize=8)
vline!([y_intercept], label="P0 (static)", linewidth=2)

MethodError: MethodError: no method matching Float64(::Vector{Float64})
Closest candidates are:
  (::Type{T})(!Matched::SymbolicUtils.Symbolic) where T<:Union{AbstractFloat, Integer, Complex{<:AbstractFloat}, Complex{<:Integer}} at ~/.julia/packages/Symbolics/UrqtQ/src/Symbolics.jl:150
  (::Type{T})(!Matched::AbstractChar) where T<:Union{AbstractChar, Number} at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/char.jl:50
  (::Type{T})(!Matched::Base.TwicePrecision) where T<:Number at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/twiceprecision.jl:255
  ...